# Astro Lab Book 2025

In [1]:
import astropy
import ccdproc
from astropy.nddata import CCDData
from astropy.time.time_helper.function_helpers import linspace
from ccdproc import ccd_process
import numpy as np
from pathlib import Path
import ccdproc as ccdp
from astropy import units as u
import matplotlib.pyplot as plt
import photutils.background as bg
from astropy.convolution import convolve
from photutils.segmentation import make_2dgaussian_kernel
from photutils.segmentation import detect_sources
from photutils.segmentation import deblend_sources
from photutils.utils import calc_total_error
from photutils.segmentation import SourceCatalog
from photutils.aperture import CircularAperture, aperture_photometry
from photutils.background import MedianBackground
import os
import glob

### Combined Bias

In [2]:
raw_data = Path('data/bias')

calibrated_biases = glob.glob(str(raw_data / '*.fit*'))

combined_bias = ccdp.combine(calibrated_biases,
                             method='average',
                             mem_limit=350e6,
                             unit='adu'
                            )

combined_bias.meta['combined'] = True
combined_bias.write(raw_data / 'combined_bias.fit', overwrite=True)

INFO: splitting each image into 11 chunks to limit memory usage to 350000000.0 bytes. [ccdproc.combiner]


### Dark Frame Cleaning

In [5]:
path_dark_raw = 'data/darks'
combined_bias = astropy.nddata.CCDData.read('data/bias/combined_bias.fit', unit='adu')

for file_path in glob.glob(os.path.join(path_dark_raw,'*.fits')):
    ccd = ccdp.CCDData.read(file_path, unit='adu')
    ccd = ccdp.subtract_bias(ccd, combined_bias)
    file_name = os.path.basename(file_path)
    ccd.write('data/darks/bias_subtracted_' + file_name, overwrite = True)

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


In [13]:
path_subtracted_dark_data = Path('data/darks')

dark_file_paths = glob.glob(str(path_subtracted_dark_data / 'bias_subtracted_*.dark.*.fits'))

combined_darks = ccdp.combine(dark_file_paths,
                             method='median',
                             mem_limit=350e6,
                             unit='adu'
                            )

combined_darks.meta['combined'] = True
combined_darks.write('data/darks/combined_darks.fit', overwrite=True)

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: splitting each image into 23 chunks to limit memory usage to 350000000.0 bytes. [ccdproc.combiner]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the uni

### Cleaning B-Filter Flats

In [23]:
combined_bias = ccdp.CCDData.read('data/bias/combined_bias.fit', unit='adu')

for file_path in glob.glob(os.path.join('data/flat_B/*.fits')):
    ccd = ccdp.subtract_bias(ccd, combined_bias)
    ccd = ccdp.CCDData.read(file_path, unit='adu')
    ccd = ccdp.subtract_bias(ccd, combined_bias)
    file_name = os.path.basename(file_path)
    ccd.write('data/flat_B/bias_subtracted_' + file_name, overwrite = True)

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


In [26]:
seconds_unit = u.s

path_flat_bias_subtracted = Path('data/flat_B')
combined_darks = ccdp.CCDData.read('data/darks/combined_darks.fit', unit='adu')

for file_path in glob.glob(os.path.join(path_flat_bias_subtracted, 'bias_subtracted_*.flat.*.fits')):
    ccd = ccdp.CCDData.read(file_path, unit='adu')
    ccd = ccdp.subtract_dark(ccd, combined_darks, exposure_time='EXPTIME', exposure_unit = seconds_unit)
    file_name = os.path.basename(file_path)
    ccd.write('data/flat_B/dark_subtracted_' + file_name, overwrite = True)

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader inste

In [29]:
def normalise(arr, mean):
    arr.data[:] = arr.data / mean
    return arr

path_flat_cleaned = 'data/flat_B'

for file_path in glob.glob(os.path.join(path_flat_cleaned, 'dark_subtracted_bias_subtracted_*.flat.*.fits')):
    ccd = ccdp.CCDData.read(file_path, unit='adu')
    ccd_mean =  np.mean(ccd)
    ccd = normalise(ccd, ccd_mean)
    file_name = os.path.basename(file_path)
    ccd.write(path_flat_cleaned + '/nomalised_' + file_name, overwrite=True)

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


In [36]:
path_norm_flats = Path('data/flat_B')
calibrated_flats = glob.glob('data/flat_B/nomalised_dark_subtracted_bias_subtracted_*.flat.*.fits')

combined_flats = ccdp.combine(calibrated_flats,
                             method='median',
                             mem_limit=350e6,
                             unit = 'adu'
                            )

combined_flats.meta['combined'] = True
combined_flats.write('data/flat_B/combined_B_flats.fit', overwrite = True)

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: splitting each image into 26 chunks to limit memory usage to 350000000.0 bytes. [ccdproc.combiner]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the uni